## environment

In [1]:
import numpy as np
import pandas as pd

from data_loader import load_names_from_web, holdout_split

import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error

In [2]:
# !pip install tensorflow

In [3]:
# torch needs to be 1.13 or higher
# pip install --upgrade torch

In [4]:
import torch
from neuralforecast import NeuralForecast
from neuralforecast.models import LSTM
from neuralforecast.losses.pytorch import DistributionLoss

In [5]:
import optuna

In [6]:
import os

In [7]:
print(torch.__version__)

2.2.2


In [8]:
# pip install neuralforecast

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.4.1 requires pyqt5<5.16, which is not installed.
spyder 5.4.1 requires pyqtwebengine<5.16, which is not installed.

Successfully installed Mako-1.3.2 aiohttp-3.9.3 aiosignal-1.3.1 alembic-1.13.1 async-timeout-4.0.3 colorlog-6.8.2 coreforecast-0.0.7 frozenlist-1.4.1 lightning-utilities-0.11.2 multidict-6.0.5 neuralforecast-1.7.0 optuna-3.6.1 pyarrow-15.0.2 pytorch-lightning-2.2.1 ray-2.10.0 tensorboardX-2.6.2.2 torch-2.2.2 torchmetrics-1.3.2 typing-extensions-4.10.0 utilsforecast-0.1.2 yarl-1.9.4
Note: you may need to restart the kernel to use updated packages.


In [9]:
'''
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.preprocessing import MinMaxScaler
'''

'\nfrom tensorflow.keras.models import Sequential\nfrom tensorflow.keras.layers import LSTM, Dense\nfrom tensorflow.keras.preprocessing.sequence import TimeseriesGenerator\nfrom sklearn.preprocessing import MinMaxScaler\n'

ref: https://forecastegy.com/posts/multiple-time-series-forecasting-with-lstm-in-python/

In [9]:
print(torch.cuda.is_available())

False


for PyTorch on my laptop (which has an M2 chip):

ref: https://www.mrdbourke.com/pytorch-apple-silicon/


## load data

In [10]:
dfraw = load_names_from_web(category="national", hide_pre_1937=True, use_existing_files=True)
dfraw.sample(5)

,state,year,name,M/F,count
1610636,US,1942,Berdina,F,6
1114174,US,1984,Skye,M,27
307508,US,2016,Chaston,M,15
890010,US,1948,Vicky,F,858
1104444,US,1984,Yaminah,F,17


## prep data
* combine name and gender
* drop state
* Do percent of births, not raw count
* min max scaler? shrinks the max perc of 2.9 to 1, and thus shrinks other numbers. Maybe not necessary or helpful.
* I think I want to fill-in missing values with zeros. That way the model will be good at predicting zeros. (Or, same idea, but make it 4s instead of 0s, **or, subtract 4 from all non-zero.** ). There will be a lot of zeros. Maybe, before minmaxscaler, apply a log transformation: add 1 then take the np.log.





In [11]:
# fill_nan_with = 0
# do_minmax, min_val, max_val = True, 0, 1

In [12]:
df1 = dfraw.copy()
df1['name_g'] = df1['name'] + '_' + df1['M/F']
df1.drop(columns=['state', 'name', 'M/F'], inplace=True)
print(df1.shape)

# rename columns to work with the neuralforecast library
df1.rename(columns={"year": "ds", "name_g": "unique_id", "count": "y"}, inplace=True)

df1.head()

(1745752, 3)


,ds,y,unique_id
0,2000,25957,Emily_F
1,2000,23085,Hannah_F
2,2000,19968,Madison_F
3,2000,17998,Ashley_F
4,2000,17712,Sarah_F


In [13]:
'''
year_count = dfraw.groupby(['year']).sum(numeric_only=True).reset_index()
df1 = pd.merge(df1, year_count, on=['year'], how='left', suffixes=('', '_agg'))
df1['perc'] = 100*df1['count']/df1['count_agg']
df1.drop(['count_agg'], axis=1, inplace=True)
df1.drop(columns=['count'], inplace=True)

scaler = MinMaxScaler(feature_range=(0, 1))
df1['perc_scaled'] = scaler.fit_transform(df1[['perc']])
df1.drop(columns=['perc'], inplace=True)

df1 = df1.pivot(index='name_g', columns='year', values='perc_scaled')
df1.fillna(fill_nan_with, inplace=True)

print(df1.shape)
df1.sample(5)
'''

"\nyear_count = dfraw.groupby(['year']).sum(numeric_only=True).reset_index()\ndf1 = pd.merge(df1, year_count, on=['year'], how='left', suffixes=('', '_agg'))\ndf1['perc'] = 100*df1['count']/df1['count_agg']\ndf1.drop(['count_agg'], axis=1, inplace=True)\ndf1.drop(columns=['count'], inplace=True)\n\nscaler = MinMaxScaler(feature_range=(0, 1))\ndf1['perc_scaled'] = scaler.fit_transform(df1[['perc']])\ndf1.drop(columns=['perc'], inplace=True)\n\ndf1 = df1.pivot(index='name_g', columns='year', values='perc_scaled')\ndf1.fillna(fill_nan_with, inplace=True)\n\nprint(df1.shape)\ndf1.sample(5)\n"

## split

In [14]:
split_year = 2020
train1 = df1.loc[df1['ds'] < split_year]
valid1 = df1.loc[df1['ds'] >=split_year]
h = valid1['ds'].nunique() # number of years to forecast / check
h

3

In [15]:

# don't train on any names where the last 1 years was zero / missing / NaN. I will assume they stay zero for now
print(train1.shape, train1['unique_id'].nunique())
ids_with_records = train1[train1['ds'].isin([split_year-1])]['unique_id'].unique()
train2 = train1[train1['unique_id'].isin(ids_with_records)]
print(train2.shape, train2['unique_id'].nunique())

# ? don't train on any names where the most of the last 10 years were zero / missing / NaN ?


(1650635, 3) 105291
(964720, 3) 32109


## model

options for ref:
* try other models besides LSTM (easy with neuralforecast)

hyperparameters:
* loss distibution and conf level

In [35]:
models = [LSTM(
    # input size,          # docd
    h=h,                   # docd: horizon: the number of steps into the future to predict
    # state_hsize:int=200, # docd
    # step_size:int=1,     # docd
    # n_layers:int=2,      # docd
    # bias:bool=True,      # docd 
    # dropout:float=0.0,   # docd
    learning_rate=1e-3,    # docd. too large: diverge.    too small: slow to converge   1e-3 is default
    # normalize:bool=True, # docd
    loss=DistributionLoss(distribution='Normal', level=[90]), # docd
    random_seed=42,        # docd
    
    max_steps=100,           # The lower the learning_rate, the more steps are necessary for convergence. high steps but low learning rate is good but takes a long time
    encoder_n_layers=2,      # too large risks overfitting
    encoder_hidden_size=200, # number of neurons in hidden layers. if too large: overfitting
    context_size=10,         # how many steps in past used to predict future
    decoder_hidden_size=200, # number of neurons of the decoder’s internal representations.
    decoder_layers=2,        # 2 be default
    scaler_type='standard',  # None, standard, robust
    # futr_exog_list=[],     # I have no exogenous variables
    # stat_exog_list=[],     # static external variables. It will automatically add the variables to the input for the forecast horizon.
    )]

Seed set to 42


In [36]:
model = NeuralForecast(models=models, freq=1) # freq: see https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases

In [37]:
model.fit(train2)

Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

Training: |                                               | 0/? [00:00<?, ?it/s]

Validation: |                                             | 0/? [00:00<?, ?it/s]

In [19]:
# p =  model.predict(futr_df=valid2).reset_index()
p =  model.predict().reset_index()

Predicting: |                                             | 0/? [00:00<?, ?it/s]

In [20]:
p = p.merge(valid1[['ds','unique_id', 'y']], on=['ds', 'unique_id'], how='left')
p.head()

,unique_id,ds,LSTM,LSTM-median,LSTM-lo-90,LSTM-hi-90,y
0,Aaban_M,2020,9.319749e-08,1.283978e-07,-1.967076e-07,2.209606e-07,NaN
1,Aaban_M,2021,-2.706414e-08,-1.239215e-07,-2.027558e-07,2.669389e-07,NaN
2,Aaban_M,2022,9.489244e-08,-1.227262e-07,-2.672379e-07,-9.180049e-08,7.0
3,Aabha_F,2020,9.146271e-08,1.334925e-07,2.663763e-07,-1.884766e-07,5.0
4,Aabha_F,2021,-2.778101e-08,-2.440391e-07,2.692279e-07,-4.971610e-08,5.0


In [21]:
p.loc[p['unique_id']=='John_M']

,unique_id,ds,LSTM,LSTM-median,LSTM-lo-90,LSTM-hi-90,y
44577,John_M,2020,9.318524e-08,-1.097147e-07,4.891606e-07,3.825585e-07,8251.0
44578,John_M,2021,-3.661657e-08,-2.999204e-08,-1.397656e-07,1.398190e-07,8175.0
44579,John_M,2022,6.319514e-08,1.776334e-07,-4.032079e-07,4.216342e-07,7930.0


In [22]:
p['LSTM_rounded'] = p['LSTM'].round()
p.sort_values(by=['LSTM_rounded'], ascending=True).head()

,unique_id,ds,LSTM,LSTM-median,LSTM-lo-90,LSTM-hi-90,y,LSTM_rounded
0,Aaban_M,2020,9.319749e-08,1.283978e-07,-1.967076e-07,2.209606e-07,NaN,0.0
64222,Maysin_M,2021,-3.374855e-08,-1.311303e-07,-3.588703e-09,8.471169e-08,NaN,-0.0
64221,Maysin_M,2020,9.503642e-08,-7.156692e-08,-1.746766e-07,-6.454076e-07,NaN,0.0
64220,Maysie_F,2022,7.607312e-08,4.022274e-08,1.152993e-07,4.340270e-07,12.0,0.0
64219,Maysie_F,2021,-4.259844e-08,-6.713084e-08,-2.038769e-08,5.280296e-08,19.0,-0.0


## tune

In [23]:
def objective(trial):
    encoder_n_layers = trial.suggest_int('encoder_n_layers', 1, 3)
    encoder_hidden_size = trial.suggest_categorical('encoder_hidden_size', [64, 128, 256])
    decoder_layers = trial.suggest_int('decoder_layers', 1, 3)
    decoder_hidden_size = trial.suggest_categorical('decoder_hidden_size', [64, 128, 256])
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
    context_size = trial.suggest_int('context_size', 1, 20)  # max 20 years
    scaler_type = trial.suggest_categorical('scaler_type', ['standard', 'robust'])

    models = [LSTM(h=h,
                   loss=DistributionLoss(distribution='Normal', level=[90]),
                   max_steps=100,
                   encoder_n_layers=encoder_n_layers,
                   encoder_hidden_size=encoder_hidden_size,
                   context_size=context_size,
                   decoder_hidden_size=decoder_hidden_size,
                   decoder_layers=decoder_layers,
                   learning_rate=learning_rate,
                   scaler_type=scaler_type,
                   # futr_exog_list=['onpromotion'],
                  )]

    model = NeuralForecast(models=models, freq=1)
    model.fit(train2)

    # p = model.predict(futr_df=valid).reset_index()
    p =  model.predict().reset_index()
    
    p = p.merge(valid1, on=['ds', 'unique_id'], how='left')

    loss = mean_absolute_error(p['y'], p['LSTM']) 

    return loss

In [32]:
os.environ['PYTORCH_ENABLE_MPS_FALLBACK'] = '1'

In [33]:
study = optuna.create_study(direction='minimize')

[I 2024-04-03 22:36:12,947] A new study created in memory with name: no-name-94c2bcfd-406c-487f-addf-2e55c2ff008a


In [34]:
study.optimize(objective, n_trials=2)   # 20-30 typically good to find good set of hyperparameters

Seed set to 1


Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

Training: |                                               | 0/? [00:00<?, ?it/s]

[W 2024-04-03 22:36:15,756] Trial 0 failed with parameters: {'encoder_n_layers': 1, 'encoder_hidden_size': 256, 'decoder_layers': 2, 'decoder_hidden_size': 128, 'learning_rate': 3.813129845657608e-05, 'context_size': 7, 'scaler_type': 'robust'} because of the following error: NotImplementedError("The operator 'aten::nanmedian.dim_values' is not currently implemented for the MPS device. If you want this op to be added in priority during the prototype phase of this feature, please comment on https://github.com/pytorch/pytorch/issues/77764. As a temporary fix, you can set the environment variable `PYTORCH_ENABLE_MPS_FALLBACK=1` to use the CPU as a fallback for this op. WARNING: this will be slower than running natively on MPS.").
Traceback (most recent call last):
  File "/Users/paulschickler/anaconda3/lib/python3.10/site-packages/optuna/study/_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/lb/1f9q5kds7h5c90s9vmqdd9qm0000gn/T/ipykernel_28348/9

NotImplementedError: The operator 'aten::nanmedian.dim_values' is not currently implemented for the MPS device. If you want this op to be added in priority during the prototype phase of this feature, please comment on https://github.com/pytorch/pytorch/issues/77764. As a temporary fix, you can set the environment variable `PYTORCH_ENABLE_MPS_FALLBACK=1` to use the CPU as a fallback for this op. WARNING: this will be slower than running natively on MPS.

In [ ]:
# best set of hyperparameters
study.best_params

In [ ]:
# best value of the loss function
study.best_value

In [ ]:
valid2.head()

## train test validation sets
* split last n years. 
* split sets of names into training, test, and validation sets.

In [ ]:
num_years_to_predict = 10
# just set fixed numbers for dataset sizes for now. start small for debugging and speed checking 
train_size, test_size, val_size = 1000, 50, 50
random_seed = 42

In [ ]:
# shuffle the indices, then get the train, test, val indices
np.random.seed(random_seed)  # this line HAS to be inb the same cell as the shuffle for repeatability
shuffled_indices = np.random.permutation(df1.index)
train_indices = shuffled_indices[:train_size]
test_indices = shuffled_indices[train_size:train_size+test_size]
val_indices = shuffled_indices[train_size+test_size:train_size+test_size+val_size]

train_df = df1.loc[train_indices].T
test_df = df1.loc[test_indices].T
val_df = df1.loc[val_indices].T

print(train_df.shape)
train_df.iloc[-5:,:7]

## (test create_dataset: whayt does it do?)

In [ ]:
def create_dataset(series, n_steps):
    X, y = [], []
    for i in range(len(series) - n_steps):
        end_ix = i + n_steps
        if end_ix > len(series)-1:
            break
        seq_x, seq_y = series[i:end_ix], series[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [ ]:
dftest = pd.DataFrame([[1,2,3],[4,5,6], [7,8,9], [10,11,12], [13,14,15]])
dftest

In [ ]:
n_steps = 2
X, y = create_dataset(dftest[0].values, n_steps)
X

In [ ]:
y

In [ ]:
X = X.reshape((X.shape[0], X.shape[1], 1))
X

In [ ]:


import numpy as np
import pandas as pd
from IPython.display import display, Markdown

import matplotlib.pyplot as plt
from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATS, NHITS
from neuralforecast.utils import AirPassengersDF

# Split data and declare panel dataset
Y_df = AirPassengersDF
Y_train_df = Y_df[Y_df.ds<='1959-12-31'] # 132 train
Y_test_df = Y_df[Y_df.ds>'1959-12-31'] # 12 test

# Fit and predict with NBEATS and NHITS models
horizon = len(Y_test_df)
models = [NBEATS(input_size=2 * horizon, h=horizon, max_steps=50),
          NHITS(input_size=2 * horizon, h=horizon, max_steps=50)]
nf = NeuralForecast(models=models, freq='M')
nf.fit(df=Y_train_df)
Y_hat_df = nf.predict(PYTORCH_ENABLE_MPS_FALLBACK=1).reset_index()

# Plot predictions
fig, ax = plt.subplots(1, 1, figsize = (20, 7))
Y_hat_df = Y_test_df.merge(Y_hat_df, how='left', on=['unique_id', 'ds'])
plot_df = pd.concat([Y_train_df, Y_hat_df]).set_index('ds')

plot_df[['y', 'NBEATS', 'NHITS']].plot(ax=ax, linewidth=2)

ax.set_title('AirPassengers Forecast', fontsize=22)
ax.set_ylabel('Monthly Passengers', fontsize=20)
ax.set_xlabel('Timestamp [t]', fontsize=20)
ax.legend(prop={'size': 15})
ax.grid()